In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Khóa luận tốt nghiệp/Paragraph_Raking/Tensorflow

/content/drive/.shortcut-targets-by-id/1T-TkLmhLIA8qodciANOIZEcC0W8CsU9V/Khóa luận tốt nghiệp/Paragraph_Raking/Tensorflow


In [ ]:
!pip install vncorenlp -q
!pip install genz-tokenize==1.2.7a2 -q

     |████████████████████████████████| 2.6 MB 8.9 MB/s 
     |████████████████████████████████| 63.9 MB 127 kB/s 
     |████████████████████████████████| 462 kB 54.5 MB/s 


In [ ]:
import tensorflow as tf
from vncorenlp import VnCoreNLP
from genz_tokenize import Tokenize
from genz_tokenize.preprocess import vncore_tokenize
from genz_tokenize.ranking import BM25
import json
import random
import string
from tqdm import tqdm
import numpy as np
import math
import pickle
from sklearn.metrics import f1_score

In [ ]:
tokenizer = Tokenize()
MAXLEN_C = 1024
MAXLEN_Q = 277

In [ ]:
vncore_jar = '/content/drive/MyDrive/Khóa luận tốt nghiệp/Paragraph_Raking/Tensorflow/VnCoreNLP/VnCoreNLP-1.1.1.jar'
vncore = VnCoreNLP(vncore_jar)

In [ ]:
label = ['ai', 'cai gi', 'con vat', 'nhu the nao', 'number', 'tai sao', 'thoi gian', 'time', 'thuc vat', 'yes no', 'location']
postag_label = ['B', 'Np', 'Nc', 'Nu', 'N', 'Ny', 'Ni', 'Nb', 'V', 'Vb', 'A', 'Ab', 'P', 'R', 'L', 'M', 'E', 'C', 'Cc', 'I', 'T', 'Y', 'Z', 'X', 'CH']

def getTypeOfWord(vncore, text):
    out = np.zeros(shape=(MAXLEN_Q, len(postag_label)), dtype = np.int64)
    pos = pos_tag(text, vncore)
    for i in range(len(pos)):
        if i == 277:
            print(pos)
        index_pos = postag_label.index(pos[i][1])
        out[i, index_pos] = 1
    return out

def pos_tag(text, vncore):
    text = text.replace('_', ' ')
    tag = vncore.pos_tag(text)
    combine = []
    for i in tag:
        combine += i
    return combine

In [ ]:
label = ['ai', 'cai gi', 'con vat', 'nhu the nao', 'number', 'tai sao', 'thoi gian', 'time', 'thuc vat', 'yes no', 'location']
postag_label = ['B', 'Np', 'Nc', 'Nu', 'N', 'Ny', 'Ni', 'Nb', 'V', 'Vb', 'A', 'Ab', 'P', 'R', 'L', 'M', 'E', 'C', 'Cc', 'I', 'T', 'Y', 'Z', 'X', 'CH']

class Embedding(tf.keras.layers.Layer):
    def __init__(self, vocab_size, dim,max_position_embeddings,  **kwargs):
        super().__init__(**kwargs)
        self.vocab_size = vocab_size
        self.dim = dim
        self.max_position_embeddings = max_position_embeddings
        self.LayerNorm = tf.keras.layers.LayerNormalization(epsilon=1e-12, name="LayerNorm")
        self.dropout = tf.keras.layers.Dropout(rate=0.1)

    def build(self, input_shape: tf.TensorShape):
        with tf.name_scope("word_embeddings"):
            self.weight = self.add_weight(
                name="weight",
                shape=[self.vocab_size, self.dim],
            )

        with tf.name_scope("position_embeddings"):
            self.position_embeddings = self.add_weight(
                name="embeddings",
                shape=[self.max_position_embeddings, self.dim],
            )
        super().build(input_shape)

    def call(self, input_ids=None):
        inputs_embeds = tf.gather(params=self.weight, indices=input_ids)
        input_shape = tf.shape(input_ids)
        position_ids = tf.expand_dims(tf.range(start=0, limit=input_shape[-1]), axis=0)
        position_embeds = tf.gather(params=self.position_embeddings, indices=position_ids)
        final_embeddings = inputs_embeds + position_embeds
        final_embeddings = self.LayerNorm(inputs=final_embeddings)
        final_embeddings = self.dropout(inputs=final_embeddings)
        return final_embeddings

class FeedFoward(tf.keras.layers.Layer):
    def __init__(self, d_model, num_class):
        super(FeedFoward, self).__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.fc1 = tf.keras.layers.Dense(d_model)
        self.dropout = tf.keras.layers.Dropout(0.2)
        self.out = tf.keras.layers.Dense(num_class, activation = 'softmax')
    
    def call(self, inputs):
        inputs = self.flatten(inputs)
        x = self.fc1(inputs)
        x = self.dropout(x)
        return self.out(x)
class ClassifyModel(tf.keras.Model):
    def __init__(self, vocab_size,max_position_embeddings, d_model, num_heads, num_class):
        super(ClassifyModel, self).__init__()
        self.embedding = Embedding(vocab_size, d_model, max_position_embeddings)
        self.layernorm = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.mha = tf.keras.layers.MultiHeadAttention(num_heads, d_model, dropout = 0.1)
        self.fc = tf.keras.layers.Dense(1)
        self.fw = FeedFoward(d_model, num_class)
    def call(self, inputs):
        x = self.embedding(inputs)
        mask = self.mask(inputs)
        out_mha = self.mha(x, x, x)
        x = self.fc(self.layernorm(x+out_mha))
        return self.fw(x)

    def mask(self, inputs):
        masks = tf.logical_not(tf.math.equal(inputs, 0))
        masks = tf.cast(masks, dtype = tf.float32)
        return masks[:, tf.newaxis, tf.newaxis, :]

model_cls = ClassifyModel(tokenizer.vocab_size(), 1000, 256, 3, len(label))

def fullConnected(d_model):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(d_model*2, activation='gelu'))
    model.add(tf.keras.layers.Dense(d_model))
    return model

class QuestionAnalys(tf.keras.Model):
    def __init__(self, d_model, maxlen, model_cls, num_heads):
        super(QuestionAnalys, self).__init__()
        self.model_cls = model_cls
        self.model_cls.trainable = False

        self.mha1 = tf.keras.layers.MultiHeadAttention(num_heads, d_model)
        self.mha2 = tf.keras.layers.MultiHeadAttention(num_heads, d_model)

        self.layerNorm1 = tf.keras.layers.LayerNormalization()
        self.layerNorm2 = tf.keras.layers.LayerNormalization()

        self.fc1 = fullConnected(d_model)
        self.fc2 = fullConnected(d_model)
        self.fc3 = fullConnected(d_model)

        self.fc_postag = tf.keras.layers.Dense(d_model)
        self.fc_type = tf.keras.layers.Dense(d_model)

        self.out1 = tf.keras.layers.Dense(1, activation = 'gelu')
        self.flatten = tf.keras.layers.Flatten()
        self.out2 = tf.keras.layers.Dense(d_model, activation = 'gelu')
        self.out3 = tf.keras.layers.Dense(maxlen, activation = 'sigmoid')

        self.dropout1 = tf.keras.layers.Dropout(0.2)
        self.dropout2= tf.keras.layers.Dropout(0.2)

    def mask(self, inputs):
        masks = tf.logical_not(tf.math.equal(inputs, 0))
        masks = tf.cast(masks, dtype = tf.float32)
        return masks[:, tf.newaxis, tf.newaxis, :]
    
    def call(self, x):
        inputs, input_postag = x
        emb = self.model_cls.embedding(inputs)
        mask = self.mask(inputs)
        out_mha = self.mha1(emb, emb, emb, attention_mask = mask)
        norm = self.layerNorm1(out_mha+emb)
        ff = self.fc1(self.dropout1(norm))

        postag = self.fc_postag(input_postag)
        out_mha = self.mha2(ff, postag, ff, attention_mask = mask)
        norm = self.layerNorm2(ff+out_mha)
        ff = self.fc2(self.dropout2(norm))

        inputs = tf.cast(inputs, dtype = tf.int32)
        out_type = tf.expand_dims(self.fc_type(self.model_cls(inputs)), axis = 1)
        out = tf.concat([ff, out_type], axis = 1)
        out = self.fc3(out)
        out = self.out1(out)
        out = self.flatten(out)
        out = self.out2(out)
        out = self.out3(out)
        return out

d_model = 256
num_heads = 6
model_analys = QuestionAnalys(d_model, MAXLEN_Q, model_cls, num_heads)
checkpoint = tf.train.Checkpoint(
        model=model_analys)
ckpt_manager = tf.train.CheckpointManager(
    checkpoint, '/content/drive/MyDrive/Khóa luận tốt nghiệp/weight_of_word/chekcpoint2', max_to_keep=5)
if ckpt_manager.latest_checkpoint:
    checkpoint.restore(ckpt_manager.latest_checkpoint)
    print('\nLatest checkpoint restored!!!\n')


Latest checkpoint restored!!!



In [ ]:
class BM25W:
    def __init__(self, model, documents, b: float = 0.75, k1: float = 1.2) -> None:
        self.b = b
        self.k1 = k1
        self.model = model

        self.num_doc = 0
        self.fieldLens = []
        self.frequency_word_in_doc = []
        self.documents = []

        for document in documents:
            frequency = {}
            document = document.lower().split()
            self.documents.append(document)
            self.num_doc += 1
            self.fieldLens.append(len(document))
            for i in document:
                try:
                    frequency[i] += 1
                except:
                    frequency[i] = 1
            self.frequency_word_in_doc.append(frequency)
        self.avgFieldLen = np.mean(self.fieldLens)

    def cal_w(self, q):
        input_ids = tokenizer(q, max_len = MAXLEN_Q, padding = True, truncation = True)['input_ids']
        input_ids = np.expand_dims(input_ids, axis = 0)
        postag= np.expand_dims(getTypeOfWord(vncore, q), axis = 0)
        self.w = {}
        pred = self.model([input_ids, postag])[0]
        for i, v in enumerate(q.split()):
            self.w[v] = pred[i]
    def cal_idf(self, q: str) -> float:
        f_q = sum([1 if q in i else 0 for i in self.documents])
        return np.log(1+(len(self.documents)-f_q+0.5+self.w[q])/(f_q+0.5))

    def get_score(self, query: str):
        query = query.lower()
        self.cal_w(query)
        query = query.split()
        scores = []
        for i, doc in enumerate(self.documents):
            score = 0
            for q in query:
                f = self.frequency_word_in_doc[i].get(q, 0)
                idf = self.cal_idf(q)
                score += idf*((f*(self.k1+1+self.w[q]))/(f+self.k1 *
                              (1-self.b+self.b*(len(doc)/self.avgFieldLen))))
            scores.append(score)
        return scores

In [ ]:
with open('train_IR.json', 'r') as f:
    data = json.load(f)

In [ ]:
titles = set(map(lambda x:x['title'], data))

In [ ]:
vncore_jar = '/content/drive/MyDrive/Khóa luận tốt nghiệp/Paragraph_Raking/Tensorflow/VnCoreNLP/VnCoreNLP-1.1.1.jar'
vncore = VnCoreNLP(vncore_jar)
def remove_punc(text):
    punc = string.punctuation
    punc = punc.replace('_', '')
    return ''.join([i for i in text if i not in punc])

In [ ]:
documents = []
queries = []
for i in titles:
    true = list(filter(lambda x: x['title']==i and x['label'], data))
    false = list(filter(lambda x: x['title']==i and x['label'] == False, data))
    for j in true:
        queries.append(vncore_tokenize(j['question'], vncore))
        doc = []
        doc.append(vncore_tokenize(j['text'], vncore))
        for f in false:
            doc.append(vncore_tokenize(f['text'], vncore))
        documents.append(doc)

In [ ]:
result = []
for d, q in zip(documents, queries):
    bm25 = BM25(documents = d)
    scores = bm25.get_score(q)
    index = np.argmax(scores)
    result.append(index)
result.count(0)/len(result)

0.8333914255838272

In [ ]:
f1_score([0]*len(result), result, average='weighted')

0.9091254752851712

In [ ]:
print([i for i in range(len(result)) if result[i] != 0])

[3, 8, 18, 19, 23, 26, 27, 35, 56, 57, 66, 68, 74, 81, 84, 85, 88, 97, 99, 100, 102, 106, 112, 114, 122, 134, 136, 138, 139, 142, 143, 148, 149, 153, 154, 158, 162, 173, 175, 176, 186, 188, 192, 195, 197, 208, 213, 216, 221, 224, 227, 230, 232, 234, 235, 236, 238, 239, 241, 243, 244, 247, 248, 249, 256, 258, 271, 281, 287, 291, 299, 303, 304, 305, 318, 322, 326, 332, 335, 341, 343, 345, 349, 352, 353, 380, 383, 386, 387, 390, 397, 399, 401, 402, 403, 407, 421, 427, 431, 432, 433, 434, 447, 457, 460, 461, 466, 468, 469, 475, 484, 485, 488, 490, 491, 492, 507, 520, 522, 525, 530, 531, 537, 539, 542, 572, 573, 574, 577, 580, 599, 603, 604, 631, 634, 647, 652, 658, 666, 680, 683, 684, 687, 689, 690, 691, 704, 708, 726, 728, 738, 742, 745, 747, 749, 750, 752, 754, 759, 763, 767, 768, 769, 773, 775, 778, 783, 790, 799, 811, 817, 818, 828, 831, 834, 837, 841, 853, 857, 884, 889, 893, 895, 896, 897, 910, 912, 913, 915, 916, 917, 942, 946, 949, 955, 963, 978, 989, 990, 1002, 1004, 1006, 1009, 1

In [ ]:
result = []
for d, q in zip(documents, queries):
    bm25 = BM25W(model_analys, documents = d)
    scores = bm25.get_score(q)
    index = np.argmax(scores)
    result.append(index)

result.count(0)/len(result)

0.8586615545486233

In [ ]:
f1_score([0]*len(result), result, average='weighted')

0.9239568682606658

In [ ]:
print([i for i in range(len(result)) if result[i] != 0])

[8, 19, 21, 23, 24, 26, 27, 56, 66, 68, 85, 88, 97, 99, 100, 102, 106, 112, 122, 134, 136, 138, 148, 173, 175, 177, 188, 192, 197, 205, 215, 227, 230, 232, 236, 239, 247, 248, 249, 256, 271, 275, 287, 290, 291, 299, 303, 304, 305, 318, 326, 332, 335, 343, 347, 349, 352, 353, 356, 368, 381, 383, 386, 387, 390, 397, 399, 401, 403, 407, 412, 421, 427, 431, 433, 460, 461, 475, 488, 490, 491, 520, 522, 525, 530, 531, 537, 539, 542, 543, 572, 574, 577, 580, 603, 604, 631, 634, 639, 652, 658, 666, 683, 684, 687, 690, 691, 704, 708, 728, 738, 742, 745, 749, 752, 754, 763, 768, 769, 773, 775, 778, 783, 790, 799, 811, 817, 828, 831, 834, 853, 875, 884, 889, 893, 910, 912, 915, 917, 919, 963, 977, 978, 985, 1002, 1004, 1006, 1014, 1016, 1037, 1038, 1056, 1064, 1065, 1066, 1068, 1074, 1076, 1078, 1080, 1083, 1087, 1092, 1115, 1116, 1134, 1143, 1148, 1154, 1155, 1210, 1220, 1222, 1223, 1224, 1239, 1247, 1287, 1299, 1300, 1303, 1304, 1305, 1317, 1328, 1337, 1342, 1355, 1365, 1377, 1382, 1387, 1395, 

In [ ]:
# [i.lower() for i in documents[19]]

In [ ]:
doc = [
     'Iran ( tiếng Ba Tư : ایران\u200e Irān [ ʔiːˈɾɒːn ] ( nghe ) ) , gọi chính_thức là nước Cộng_hoà Hồi_giáo Iran ( tiếng Ba Tư : جمهوری_اسلامی ایران\u200e Jomhuri-ye Eslāmi-ye Irān_Về âm_thanh nàyphát âm ( trợ_giúp · thông_tin ) ) , [ 14 ] là một quốc_gia có chủ_quyền tại Tây_Á . [ 15 ] [ 16 ] Iran có biên_giới về phía tây bắc với Armenia , Azerbaijan , và Cộng_hoà Artsakh tự_xưng ; phía bắc giáp biển Caspi ; phía đông bắc giáp Turkmenistan ; phía đông giáp Afghanistan và Pakistan ; phía nam giáp vịnh Ba Tư và vịnh Oman ; còn phía tây giáp Thổ_Nhĩ_Kỳ và Iraq ( Khu_vực Kurdistan )',
     'Châu_Nam_Cực là lục_địa nằm xa nhất về phía nam của Trái_Đất , chứa cực Nam địa_lý và nằm trong vùng Nam_Cực của Nam_bán_cầu , gần như hoàn_toàn ở trong vòng Nam_Cực và được bao quanh bởi Nam_Băng_Dương . Với diện_tích 14 triệu km2 ( 5,4 triệu dặ m2 ) , châu Nam_Cực là lục_địa lớn thứ năm về diện_tích sau châu_Á , châu_Phi , Bắc_Mỹ , và Nam_Mỹ . Khoảng 98% châu Nam_Cực bị bao_phủ bởi một lớp băng có bề dày trung_bình 1,9 km ( 1,2 dặm ) . [ 3 ] Băng trải rộng ra khắp mọi phía , xa nhất lên phía bắc tới điểm cực Bắc của bán_đảo Nam_Cực .',
    'Vịnh Hạ_Long 1994 ( tái công_nhận : 2000 , 2011 ) ; Vịnh Hạ_Long nằm trong Vịnh Bắc_Bộ là một quần_thể gồm hơn 1.600 đảo lớn_nhỏ , tạo nên một phong_cảnh tuyệt đẹp giữa biển với những cột đá_vôi nhô lên . Hầu_hết những hòn đảo đều không có người và không có sự tác_động của con_người do đặc_tính dốc của chúng . Ngoài vẻ đẹp kỳ_diệu , vịnh Hạ_Long còn sở_hữu hệ_sinh_thái đặc_sắc . [ 7 ]',
    'Núi Chứa_Chan hay còn gọi núi Gia_Lào , cách TP. Hồ_Chí_Minh khoảng 110km , núi có độ cao 800m so với mực nước_biển , thuộc huyện Xuân_Lộc , tỉnh Đồng_Nai . Đây là ngọn núi cao thứ hai khu_vực Nam_Bộ .',
    'Đảo Ilha da Queimada_Grande rộng 45 ha , nằm lẻ_loi ở Nam_Đại_Tây_Dương , thuộc quyền_sở_hữu của Brazil , cách bờ biển Sao Paulo khoảng 35 km .',
    'Quốc triều hình_luật trong cuốn sách A. 341 có 13 chương , ghi_chép trong 6 quyển ( 5 quyển có 2 chương / quyển và 1 quyển có 3 chương ) , gồm 722 điều .',
    'Lưỡng quốc_trạng nguyên Nguyễn_Đăng_Đạo',
]

In [ ]:
bm25 = BM25(documents = doc)
scores = bm25.get_score(queries[3])
print(scores)

[2.180161811695985, 1.7509874199080921, 0.7338122138746307, 1.918673138629, 2.563141639925812, 0.0, 0.0]


In [ ]:
bm25 = BM25W(model_analys, documents = doc)
scores = bm25.get_score(queries[3])
scores = [i.numpy() for i in scores]
print(scores)

[4.498112, 2.1274369, 1.034862, 2.0946393, 3.0096583, 0.0, 0.0]
